In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix
)

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.ensemble import RandomForestClassifier

# Load ML-ready dataset
BASE = "../data/processed/"
df = pd.read_csv(BASE + "final_featured.csv")

df.head()

C:\Users\shash\AppData\Local\Temp\ipykernel_6532\456206890.py:17: DtypeWarning: Columns (0,1,3,7,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(BASE + "final_featured.csv")


,component,event_template,event_type,ip,is_login_attempt,is_success,label,level,pid,raw_message,...,timestamp,total_bytes,total_packets,hour,day,weekday,minute,total_attempts,total_success,total_failed
0,0,workerEnv.init() ok <*>,E2,0,0.0,0.0,0.0,notice,0.0,workerEnv.init() ok /etc/httpd/conf/workers2.p...,...,2005-12-04 04:47:44,0.0,0.0,4.0,4.0,6.0,47.0,0.0,0.0,0.0
1,0,mod_jk child workerEnv in error state <*>,E3,0,0.0,0.0,0.0,error,0.0,mod_jk child workerEnv in error state 6,...,2005-12-04 04:47:44,0.0,0.0,4.0,4.0,6.0,47.0,0.0,0.0,0.0
2,0,jk2_init() Found child <*> in scoreboard slot <*>,E1,0,0.0,0.0,0.0,notice,0.0,jk2_init() Found child 6725 in scoreboard slot 10,...,2005-12-04 04:51:08,0.0,0.0,4.0,4.0,6.0,51.0,0.0,0.0,0.0
3,0,jk2_init() Found child <*> in scoreboard slot <*>,E1,0,0.0,0.0,0.0,notice,0.0,jk2_init() Found child 6726 in scoreboard slot 8,...,2005-12-04 04:51:09,0.0,0.0,4.0,4.0,6.0,51.0,0.0,0.0,0.0
4,0,jk2_init() Found child <*> in scoreboard slot <*>,E1,0,0.0,0.0,0.0,notice,0.0,jk2_init() Found child 6728 in scoreboard slot 6,...,2005-12-04 04:51:09,0.0,0.0,4.0,4.0,6.0,51.0,0.0,0.0,0.0


In [2]:
df_labeled = df[df['source'] == 'unsw'].copy()

X = df_labeled.drop(columns=['label', 'source', 'event_type', 'raw_message', 'timestamp'])
y = df_labeled['label']

X.head(), y.head()


(     component event_template ip  is_login_attempt  is_success level  pid  \
 4000         0              0  0               0.0         0.0     0  0.0   
 4001         0              0  0               0.0         0.0     0  0.0   
 4002         0              0  0               0.0         0.0     0  0.0   
 4003         0              0  0               0.0         0.0     0  0.0   
 4004         0              0  0               0.0         0.0     0  0.0   
 
       total_bytes  total_packets  hour  day  weekday  minute  total_attempts  \
 4000        430.0           10.0   0.0  0.0      0.0     0.0             0.0   
 4001      42748.0           52.0   0.0  0.0      0.0     0.0             0.0   
 4002      13550.0           24.0   0.0  0.0      0.0     0.0             0.0   
 4003       1398.0           24.0   0.0  0.0      0.0     0.0             0.0   
 4004        802.0           16.0   0.0  0.0      0.0     0.0             0.0   
 
       total_success  total_failed  
 4000

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [4]:
from sklearn.ensemble import IsolationForest

iso = IsolationForest(
    n_estimators=200,
    contamination='auto',
    random_state=42,
    n_jobs=-1
)

iso.fit(X_train)

# Predict: returns -1 for anomaly, +1 for normal
iso_pred = iso.predict(X_test)

# Convert to 0/1
iso_pred = np.where(iso_pred == -1, 1, 0)

iso_pred[:20]

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [5]:
print("Isolation Forest Results:")
print("Precision :", precision_score(y_test, iso_pred))
print("Recall    :", recall_score(y_test, iso_pred))
print("F1 Score  :", f1_score(y_test, iso_pred))
print("ROC-AUC   :", roc_auc_score(y_test, iso_pred))

cm = confusion_matrix(y_test, iso_pred)
cm

Isolation Forest Results:
Precision : 0.3399915599943733
Recall    : 0.06750830935955088
F1 Score  : 0.11264914243102163
ROC-AUC   : 0.39411129753691826


array([[12108,  4692],
       [33386,  2417]])

In [6]:
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination='auto',
    novelty=True  # IMPORTANT: allows scoring on test set
)

# Fit LOF on train set
lof.fit(X_train)

# Predict
lof_pred = lof.predict(X_test)

# Convert -1 = anomaly → 1, +1 → 0
lof_pred = np.where(lof_pred == -1, 1, 0)

lof_pred[:20]

c:\Users\shash\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
print("LOF Results:")
print("Precision :", precision_score(y_test, lof_pred))
print("Recall    :", recall_score(y_test, lof_pred))
print("F1 Score  :", f1_score(y_test, lof_pred))
print("ROC-AUC   :", roc_auc_score(y_test, lof_pred))

cm = confusion_matrix(y_test, lof_pred)
cm

LOF Results:
Precision : 0.7781397032211365
Recall    : 0.06005083372901712
F1 Score  : 0.11149717367629518
ROC-AUC   : 0.5117813692454609


array([[16187,   613],
       [33653,  2150]])

In [8]:
from sklearn.svm import OneClassSVM

ocsvm = OneClassSVM(
    kernel='rbf',
    gamma='scale',
    nu=0.05 # proportion of anomalies (can tune)
)

# Fit on the training data (only X_train)
ocsvm.fit(X_train)

# Predict on X_test
ocsvm_pred = ocsvm.predict(X_test)

# Convert -1 → anomaly (1), +1 → normal (0)
ocsvm_pred = np.where(ocsvm_pred == -1, 1, 0)

ocsvm_pred[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0])

In [9]:
print("One-Class SVM Results:")
print("Precision :", precision_score(y_test, ocsvm_pred))
print("Recall    :", recall_score(y_test, ocsvm_pred))
print("F1 Score  :", f1_score(y_test, ocsvm_pred))
print("ROC-AUC   :", roc_auc_score(y_test, ocsvm_pred))

cm = confusion_matrix(y_test, ocsvm_pred)
cm

One-Class SVM Results:
Precision : 0.44752475247524753
Recall    : 0.031561600983157835
F1 Score  : 0.058964725527029846
ROC-AUC   : 0.47426294334872177


array([[15405,  1395],
       [34673,  1130]])

In [10]:
df_labeled = df[df['source'] == 'unsw'].copy()

X = df_labeled.drop(columns=['label', 'source', 'event_type', 'raw_message', 'timestamp'])
y = df_labeled['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [11]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)

In [12]:
rf_pred = rf.predict(X_test)

print("Random Forest Results:")
print("Precision :", precision_score(y_test, rf_pred))
print("Recall    :", recall_score(y_test, rf_pred))
print("F1 Score  :", f1_score(y_test, rf_pred))
print("ROC-AUC   :", roc_auc_score(y_test, rf_pred))

cm = confusion_matrix(y_test, rf_pred)
cm

Random Forest Results:
Precision : 0.9469978271034464
Recall    : 0.9616512582744463
F1 Score  : 0.9542682926829268
ROC-AUC   : 0.9234744386610327


array([[14873,  1927],
       [ 1373, 34430]])